In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data1 = pd.read_csv('result/rsrp-l900-alljabo-40x40-pop.csv/0.part')
data2 = pd.read_csv('result/rsrp-l900-alljabo-40x40.csv/0.part')

existing_sites = pd.read_excel('sites/Existing Sites-L900.xlsx')
new_sites = pd.read_csv('sites/1657 Sites.csv')

kecamatan = gpd.read_file('Polygon XL - EID Area/Kecamatan_Jabotabek.TAB')

In [3]:
existing_sites = existing_sites[['Tower ID', 'Frequency System', 'Longitude', 'Latitude']]
new_sites = new_sites[['Planning ID','Lat', 'Long']]

In [4]:
data1.rename(columns={'rsrp-l900':'rsrp-l900-qty'}, inplace=True)

In [5]:
data1

,geometry_polygon,rsrp-l900-qty
0,"POLYGON ((105.280265 -6.568783, 105.280624 -6....",1.0
1,"POLYGON ((105.318721 -6.481449, 105.31908 -6.4...",1.0
2,"POLYGON ((105.31908 -6.480011, 105.319439 -6.4...",1.0
3,"POLYGON ((105.31908 -6.480371, 105.319439 -6.4...",1.0
4,"POLYGON ((105.31908 -6.48073, 105.319439 -6.48...",1.0
...,...,...
1553212,"POLYGON ((107.64296 -6.200398, 107.64332 -6.20...",2.0
1553213,"POLYGON ((107.64332 -6.120971, 107.643679 -6.1...",1.0
1553214,"POLYGON ((107.64332 -6.148645, 107.643679 -6.1...",2.0
1553215,"POLYGON ((107.64332 -6.149723, 107.643679 -6.1...",1.0


In [6]:
data2.rename(columns={'geometry_polygon': 'geometry_polygon1'}, inplace=True)
data1 = pd.concat([data1, data2], axis=1, join='inner')

In [7]:
data1['compare'] = data1['geometry_polygon'] == data1['geometry_polygon1']
data1.reset_index(drop=True, inplace=True)
data1.drop(columns=['geometry_polygon1'], inplace=True)
data1 = data1[data1['rsrp-l900-qty'] > 150] 
data1 = data1[data1['rsrp-l900'] < -105]

In [8]:
data1   = gpd.GeoDataFrame(data1, geometry=gpd.GeoSeries.from_wkt(data1['geometry_polygon']), crs=4326)
existing_sites = gpd.GeoDataFrame(existing_sites, geometry=gpd.points_from_xy(existing_sites['Longitude'], existing_sites['Latitude']), crs=4326)
new_sites = gpd.GeoDataFrame(new_sites, geometry=gpd.points_from_xy(new_sites['Long'], new_sites['Lat']), crs=4326)

In [9]:
data1 = data1.sjoin_nearest(existing_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_existing')
data1.drop(columns=['index_right'], inplace=True)
data1 = data1.sjoin_nearest(new_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_new')

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [10]:
data1.reset_index(inplace=True)

In [11]:
data1.drop(columns=['index'], inplace=True)

In [12]:
data1.rename(columns={'rsrp-l900':'rsrp-l900-avg'}, inplace=True)

In [13]:
data1.drop(columns=['index_right', 'geometry_polygon'], inplace=True)

In [14]:
data1 = gpd.sjoin(left_df= data1, right_df=kecamatan, how='left', predicate='within')

In [15]:
data1.to_csv('result/result-badgrid-l900.csv')